# SIMULATION CAPACITOR DISCHARGE WELDING - Multiphysik

>ansys process
- Working Directory
- Input Parameters
- Geometry
- Mechanical Feld Setup
- Thermal Electric Feld Setup
- Solution Mechanical Problem
- Solution Thermal Electric Problem
- Plot Result
- Evaluation

### Bibliotheken importieren

In [ ]:
# Importiere benötigte Bibliotheken
import os
import pyansys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MultipleLocator
from IPython.display import Image
import time

In [ ]:
# ANSYS Prozess beenden
os.system("taskkill /f /im ANSYS.exe")

In [ ]:
def SaveAndExit():
    ansys.save()
    ansys.Exit()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Working Directory
TODO:
- change cwd

In [ ]:
# cwd change
def Newdir(path):
    import os
    # new dir
    try:
        os.makedirs(path)
    except FileExistsError:
        print(path + '\t directory already exists!')
    # cwd
    os.chdir(path)
    print('cwd:', os.getcwd())

In [ ]:
# ansys apdl aktivieren
filecode = 'therm_cdw_ring_al_py2'  # TODO
cwd = 'E:/Lu/SHK/Pyansys/Solution/'  # TODO
mkpath = cwd + filecode
Newdir(mkpath)
ansys = pyansys.Mapdl(run_location=os.getcwd(),
                      jobname=filecode,
                      interactive_plotting=True)

### Input Parameters
TODO:  

- Mat path 
- Puls name and 
- Puls path 
- sep = ',' or '\t'
- Force rate  entry
- Ignition force

In [ ]:
# Subfolder for output data
subfolder = 'Output'
sub_path = os.path.join(ansys.path,subfolder)
try:
    os.makedirs(sub_path)
except FileExistsError:
    print (sub_path + '\t directory for output already exists!')

# mat path
mat_path =  'E:/Lu/SHK/Material'                     # TODO
mat_sheet = 'ENAW5083_JMAT_EDIT'
mat_buckel ='ENAW6082_JMAT_EDIT'
mat_elektrode = 'CUCR1ZR_WSK'

# current path
puls_name = 'I_NIMAK_0063'                     # TODO
puls_path =  'E:/Lu/SHK/Pyansys/Puls/'                              # TODO
puls = pd.read_csv(puls_path + puls_name + '.csv', header = None,sep=',')           #TODO

# physiscs name
PHYS_STRUCT       = 'Struct'
PHYS_T_E          = 'T_E'
STRUCT_FILE       = 'Structure'
T_E_FILE          = 'Thermo_Electric'

# time data
tm_lines = len(puls)
tmax_i = max(puls.iloc[:,0])
tmin_i = min(puls.iloc[:,0])
tinc_i = (tmax_i - tmin_i)/(tm_lines-1)
TIME_PF_ABS    = 1         # Time span for applying pre force      
TIME_SP        = 1              # Spare time bt end of pre force application and start of weld force application

# geometrie parameters
PROJECTION_H1     = 0.0050
PROJECTION_H2     = 0.0013
PROJECTION_L1     = 0.0005
PROJECTION_R1     = 0.020/2
PROJECTION_R2     = 0.0140/2
PROJECTION_R3     = 0.0034
PROJECTION_PHI_1  = 30
PROJECTION_PHI_2  = 30
SHEET_R           = PROJECTION_R1
SHEET_H           = 0.0020
ELECTRODE_R1      = 0.020/2
ELECTRODE_R2      = 0.0075
ELECTRODE_R3      = 0.0069
ELECTRODE_H1      = 0.0220
ELECTRODE_H2      = 0.0100
PI = np.arccos(-1)

# constants setup
T_UNI             = 298
LRNTZ_NMBR        = 2.44E-8
CNTCT1_LYR_THCKNSS= 0.00005
CNTCT1_SRFC_CNDTN = 1

# input data for mechanical loads
IN_F_PRE    = 20E3                         	                                                        # Pre load force [N]
IN_F_W_DT  = float(input('Force rate 25E3 or 100E3 [N/s]: '))        # Force rate [N/s] TODO
IN_F_IGN    = 22.9E3                                                                               # Ignition force [N] TODO
IN_F_FIN   = IN_F_IGN+IN_F_W_DT*tmax_i                                      # Final force [N] (end of current flow)
TIME_IGN = (IN_F_IGN-IN_F_PRE)/IN_F_W_DT                 # Time bt start of weld force application and ignition
TIME_SWF_ABS   = TIME_SP+TIME_PF_ABS      	# Starting time for application of welding force (absolute)
TIME_SI_ABS   = TIME_SWF_ABS+TIME_IGN      # Starting time of current flow (absolute)
TIME_E_ABS  = TIME_SI_ABS+tmax_i         # End time of simulation (absolute)

# 4 arts of timestep for each mechanical loadsteps
DT_PF  = TIME_PF_ABS/10                 # Time step size for pre loading
DT_SP  = TIME_SP                        # Time step size for spare time
DT_WF_SI = TIME_IGN/10   # Time step size for mechanical part of simulation
DT_I  = .1E-3  # Time step size for time span of current flow

# current time edit and resave to file
puls.iloc[:,0] = puls.iloc[:,0] + TIME_SI_ABS
puls_name = 'NIMAK63_I_EDIT'
puls.to_csv(puls_path + puls_name+ '.csv',header=None,index=None)
print('Zeitschrittweit: ', tinc_i)
print('first 5 values of puls')
puls.head()

In [ ]:
TIME_SI_ABS

In [ ]:
# weite puls to ansys table and array
ansys.dim('TABLE_I', 'table', tm_lines, 1, 1, 'time')
ansys.tread('TABLE_I', puls_path + puls_name, 'csv')
ansys.dim('ARRAY_I', 'array', tm_lines + 1, 2)
ansys.mfun('ARRAY_I(1,1)', 'copy', 'TABLE_I(0,0)')

### Geometry

In [ ]:
ansys.prep7()
# sheet
ansys.k(100,0,0)
ansys.k(101,PROJECTION_R3,0)
if PROJECTION_PHI_1 == 90:
    ansys.k(102,PROJECTION_R2-PROJECTION_L1/2,0)
else:
    ansys.k(102,PROJECTION_R2-PROJECTION_L1/2-PROJECTION_H2/(np.tan(PROJECTION_PHI_1*PI/180)),0)
if PROJECTION_PHI_2 == 90 : 
    ansys.k(103,PROJECTION_R2+PROJECTION_L1/2,0)    
else:
    ansys.k(103,PROJECTION_R2+PROJECTION_L1/2+PROJECTION_H2/(np.tan(PROJECTION_PHI_2*PI/180)),0)

ansys.k(104,PROJECTION_R1,0)
ansys.k(105,PROJECTION_R1,-SHEET_H)
if PROJECTION_PHI_1 == 90:
    ansys.k(106,PROJECTION_R2-PROJECTION_L1/2,-SHEET_H)
else:
    ansys.k(106,PROJECTION_R2-PROJECTION_L1/2-PROJECTION_H2/(np.tan(PROJECTION_PHI_1*PI/180)),-SHEET_H)
if PROJECTION_PHI_2 == 90:
    ansys.k(107,PROJECTION_R2+PROJECTION_L1/2,-SHEET_H)
else:
    ansys.k(107,PROJECTION_R2+PROJECTION_L1/2+PROJECTION_H2/(np.tan(PROJECTION_PHI_2*PI/180)),-SHEET_H)

ansys.k(108,PROJECTION_R3,-SHEET_H)
ansys.k(109,0,-SHEET_H)

# Buckel
ansys.k(200,PROJECTION_R3,PROJECTION_H1)
ansys.k(201,PROJECTION_R1,PROJECTION_H1)
ansys.k(202,PROJECTION_R1,PROJECTION_H2+PROJECTION_H1*0.30)
ansys.k(203,PROJECTION_R1,PROJECTION_H2+PROJECTION_H1*0.10)
ansys.k(204,PROJECTION_R1,PROJECTION_H2)
if PROJECTION_PHI_2 == 90:
    ansys.k(205,PROJECTION_R2+PROJECTION_L1/2,PROJECTION_H2)
else:
    ansys.k(205,PROJECTION_R2+PROJECTION_L1/2+PROJECTION_H2/(np.tan(PROJECTION_PHI_2*PI/180)),PROJECTION_H2)
ansys.k(206,PROJECTION_R2+PROJECTION_L1/2,0)
ansys.k(207,PROJECTION_R2-PROJECTION_L1/2,0)
if PROJECTION_PHI_1 == 90:
    ansys.k(208,PROJECTION_R2-PROJECTION_L1/2,PROJECTION_H2)
else:
    ansys.k(208,PROJECTION_R2-PROJECTION_L1/2-PROJECTION_H2/(np.tan(PROJECTION_PHI_1*PI/180)),PROJECTION_H2)
ansys.k(209,PROJECTION_R3,PROJECTION_H2)
ansys.k(210,PROJECTION_R3,PROJECTION_H2+PROJECTION_H1*0.10)
ansys.k(211,PROJECTION_R3,PROJECTION_H2+PROJECTION_H1*0.30)

# Lower Electrode
ansys.k(300,0,-SHEET_H)
ansys.k(301,ELECTRODE_R1,-SHEET_H)
ansys.k(302,ELECTRODE_R1,-SHEET_H-0.25*ELECTRODE_H2)
ansys.k(303,0,-SHEET_H-0.25*ELECTRODE_H2)
ansys.k(310,0,-SHEET_H-0.25*ELECTRODE_H2)
ansys.k(311,ELECTRODE_R1,-SHEET_H-0.25*ELECTRODE_H2)
ansys.k(312,ELECTRODE_R1,-SHEET_H-ELECTRODE_H1)
ansys.k(313,ELECTRODE_R2,-SHEET_H-ELECTRODE_H1)
ansys.k(314,ELECTRODE_R3,-SHEET_H-ELECTRODE_H2)
ansys.k(315,0,-SHEET_H-ELECTRODE_H2)

#  Upper Electrode
ansys.k(400,0,PROJECTION_H1+0.25*ELECTRODE_H2)
ansys.k(401,ELECTRODE_R1,PROJECTION_H1+0.25*ELECTRODE_H2)
ansys.k(402,ELECTRODE_R1,PROJECTION_H1)
ansys.k(403,0,PROJECTION_H1)
ansys.k(413,0,PROJECTION_H1+0.25*ELECTRODE_H2)
ansys.k(412,ELECTRODE_R1,PROJECTION_H1+0.25*ELECTRODE_H2)
ansys.k(411,ELECTRODE_R1,PROJECTION_H1+ELECTRODE_H1)
ansys.k(410,ELECTRODE_R2,PROJECTION_H1+ELECTRODE_H1)
ansys.k(415,ELECTRODE_R3,PROJECTION_H1+ELECTRODE_H2)
ansys.k(414,0,PROJECTION_H1+ELECTRODE_H2)

In [ ]:
# Area
ansys.numstr('area', 100)
ansys.numstr('line', 100)
ansys.a(100, 101, 108, 109)
ansys.a(101, 102, 106, 108)
ansys.a(102, 103, 107, 106)
ansys.a(103, 104, 105, 107)
ansys.aglue(100, 101, 102, 103)
ansys.numstr('area', 200)
ansys.numstr('line', 200)
ansys.a(205, 206, 207, 208)
ansys.a(203, 204, 209, 210)
ansys.a(202, 203, 210, 211)
ansys.a(201, 202, 211, 200)
ansys.aglue(200, 201, 202, 203)
ansys.numstr('area', 300)
ansys.numstr('line', 300)
ansys.a(300, 301, 302, 303)
ansys.a(310, 311, 312, 313, 314, 315)
ansys.aglue(300, 301)
ansys.numstr('area', 400)
ansys.numstr('line', 400)
ansys.a(400, 401, 402, 403)
ansys.a(410, 411, 412, 413, 414, 415)
ansys.aglue(400, 401)
# area plot and wait one second (optional)
ansys.pnum('line', 1)
ansys.number(0)
ansys.aplot()

In [ ]:
Image(filename='{}/{}000.png'.format(mkpath,filecode),height=600,width = 600)

### Mechanical Feld Setup
- Material
- Meshing
- Contact Elements
- Bondary Conditions
- Write Physics

In [ ]:
def mat_read(matnumber, matname, mat_path):
    ansys.prep7()
    ansys.mat(matnumber)
    ansys.input(matname, 'si_mpl', mat_path)
    time.sleep(1)


mat_read(1, mat_sheet, mat_path)
mat_read(2, mat_buckel, mat_path)
mat_read(3, mat_elektrode, mat_path)

In [ ]:
ansys.mplist('all')

In [ ]:
# definition of parameters to save the tempnumber
NTEMP_MAT1 = 'NTEMP_MAT1'
NTEMP_MAT2 = 'NTEMP_MAT2'
NTRSVX_MAT1 = 'NTRSVX_MAT1'
NTRSVX_MAT2 = 'NTRSVX_MAT2'
# definition of paramters to save the tablename
TABLE_YIELD_MAT1 = 'TABLE_YIELD_MAT1'
TABLE_YIELD_MAT2 = 'TABLE_YIELD_MAT2'
TABLE_RSVX_MAT1 = 'TABLE_RSVX_MAT1'
TABLE_RSVX_MAT2 = 'TABLE_RSVX_MAT2'

# get tempnumber of each Mat
ansys.get(NTEMP_MAT1, 'MISO', 1, 'NTEMP')
ansys.get(NTEMP_MAT2, 'MISO', 2, 'NTEMP')
ansys.get(NTRSVX_MAT1, 'RSVX', 1, 'NTEMP')
ansys.get(NTRSVX_MAT2, 'RSVX', 2, 'NTEMP')

# set up ansys table to save the data
ansys.dim(TABLE_YIELD_MAT1, 'table', 'NTEMP_MAT1', 1, 1, 'temp')
ansys.dim(TABLE_YIELD_MAT2, 'table', 'NTEMP_MAT2', 1, 1, 'temp')
ansys.dim(TABLE_RSVX_MAT1, 'table', 'NTRSVX_MAT1', 1, 1, 'temp')
ansys.dim(TABLE_RSVX_MAT2, 'table', 'NTRSVX_MAT2', 1, 1, 'temp')

In [ ]:
ansys.load_parameters()
ansys.parameters

In [ ]:
def Matdata_in_Table(NTMEP, MPlab, Matnum, TB, YIELD=True):
    for i in range(int(ansys.parameters[NTMEP])):
        i += 1
        ansys.get('TEMP_', MPlab, Matnum, 'TVAL', i)
        ansys.run('{}({},0) = TEMP_'.format(TB, i))
        if (YIELD == True):
            ansys.get('Matdata', MPlab, Matnum, 'TEMP', 'TEMP_', 'CONST', 2)
        else:
            ansys.get('Matdata', MPlab, Matnum, 'TEMP', 'TEMP_', 'CONST', i)
        ansys.run('{}({},1) = Matdata'.format(TB, i))
    ansys.cfopen('{}/{}/{}'.format(mkpath, subfolder, TB), 'csv')
    with ansys.non_interactive:
        ansys.run("*VWRITE,{}(1,0),{}(1,1)".format(TB, TB))
        ansys.run("(ES9.2,',',ES10.3)")
    ansys.finish()
    time.sleep(1)


Matdata_in_Table(NTEMP_MAT1, 'MISO', 1, TABLE_YIELD_MAT1, True)
Matdata_in_Table(NTEMP_MAT2, 'MISO', 2, TABLE_YIELD_MAT2, True)
Matdata_in_Table(NTRSVX_MAT2, 'RSVX', 2, TABLE_RSVX_MAT2, False)
Matdata_in_Table(NTRSVX_MAT1, 'RSVX', 1, TABLE_RSVX_MAT1, False)

In [ ]:
df = pd.read_csv('{}/{}/{}.csv'.format(mkpath,subfolder,TABLE_YIELD_MAT2),header=None,sep = ',' ,engine='python',names = ['TEMP','YIELD'])
df.head(10)

In [ ]:
### delete mat if something wrong happened
# ansys.prep7()
# ansys.mpdele('all','all')
# ansys.tbdele('all','all')
# ansys.mptemp()

#### Meshing

In [ ]:
# Elements und Mat zuweisen
ansys.prep7()
ansys.et(1,183,0,'',1)
ansys.asel('s','','',100,104,1)
ansys.aatt(1,'',1)
ansys.asel('s','','',200,204,1)
ansys.aatt(2,'',1)
ansys.asel('s','','',300,302,2)
ansys.asel('a','','',400,402,2)
ansys.aatt(3,'',1)
ansys.allsel('all')
# Meshing
ansys.mopt('LSMO','on')     
ansys.esize(0.0002)
ansys.allsel('all')
ansys.asel('s','','',102,200,98)
ansys.aesize('all',0.00006)
ansys.asel('s','','',101,103,2)
ansys.asel('a','','',204)
ansys.aesize('all',0.00008)
ansys.asel('s','','',100,202,102)
ansys.aesize('all',0.0001)
ansys.allsel('all')
ansys.amesh('all')
# plot elements
ansys.pnum('mat',1)
ansys.number(1)
ansys.eplot()
# element shape quality report
ansys.prep7()
ansys.shpp('SUMM')

In [ ]:
Image(filename='{}/{}001.png'.format(mkpath,filecode),height=600,width = 600)

#### Contact Elements
- contact pair 1: ring - lower sheer'

In [ ]:
ansys.prep7()
ansys.mat(2)
ansys.r(3)
ansys.real(3)
ansys.et(2,169)
ansys.et(3,172)
ansys.r(3,'',1.0,0.1,0,)
ansys.rmore('','',1.0E20,0.0,1.0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.keyopt(3,1,0)
ansys.keyopt(3,2,4)
ansys.keyopt(3,3,0)
ansys.keyopt(3,4,0)
ansys.keyopt(3,5,0)
ansys.keyopt(3,6,2)
ansys.keyopt(3,7,0)
ansys.keyopt(3,8,0)
ansys.keyopt(3,9,0)
ansys.keyopt(3,10,0)
ansys.keyopt(3,11,0)
ansys.keyopt(3,12,0)
ansys.nlgeom('on')
# GENERATE THE TARGET SURFACE
ansys.lsel('s','','',107)
ansys.type(2)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()
# GENERATE THE CONTACT SURFACE
ansys.lsel('s','','',200,202,1)
ansys.type(3)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()
ansys.allsel('all')

- contact pair 2: upper electrode - sheet

In [ ]:
ansys.prep7()
ansys.mat(1)
ansys.r(5)
ansys.real(5)
ansys.et(4,169)
ansys.et(5,172)
ansys.r(5,'',1.0,0.1,0,)
ansys.rmore('','',1.0E20,0.0,1.0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.keyopt(5,1,0)
ansys.keyopt(5,2,0)
ansys.keyopt(5,3,0)
ansys.keyopt(5,4,0)
ansys.keyopt(5,5,0)
ansys.keyopt(5,6,2)
ansys.keyopt(5,7,0)
ansys.keyopt(5,8,0)
ansys.keyopt(5,9,0)
ansys.keyopt(5,10,0)
ansys.keyopt(5,11,0)
ansys.keyopt(5,12,5)
ansys.nlgeom('on')

ansys.lsel('s','','',213)
ansys.type(4)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()

ansys.lsel('s','','',402)
ansys.type(5)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()

ansys.allsel('all')

- contact pair 3: lower electrode - sheet  

In [ ]:
ansys.prep7()
ansys.mat(3)
ansys.r(7)
ansys.real(7)
ansys.et(6,169)
ansys.et(7,172)
ansys.r(7,'',1.0,0.1,0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.keyopt(7,1,0)
ansys.keyopt(7,2,0)
ansys.keyopt(7,3,0)
ansys.keyopt(7,4,0)
ansys.keyopt(7,5,0)
ansys.keyopt(7,6,2)
ansys.keyopt(7,7,0)
ansys.keyopt(7,8,0)
ansys.keyopt(7,9,0)
ansys.keyopt(7,10,0)
ansys.keyopt(7,11,0)
ansys.keyopt(7,12,5)
ansys.nropt('unsym')
ansys.nlgeom('on')

ansys.lsel('s','','',300)
ansys.type(6)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()

ansys.lsel('s','','',106,112,3)
ansys.lsel('a','','',102)
ansys.type(7)
ansys.nsll('s',1)
ansys.esln('s',0)
ansys.esurf()

ansys.allsel('all')

In [ ]:
# check the contact definition
ansys.cncheck()
ansys.elist()
# contact pair plot 
ansys.esel('s','ename','',172)
ansys.esel('a','ename','',169)
ansys.replot()
ansys.allsel('all')

In [ ]:
Image(filename='{}/{}002.png'.format(mkpath,filecode),height=600,width = 600)

#### Boundary Conditions 

In [ ]:
ansys.allsel('all')
ansys.nsel('s','loc','x',0)
ansys.d('all','',0,'','','','ux')
ansys.lsel('s','','',404)
ansys.nsll('s',1)
ansys.cp(1,'uy','all')
ansys.lsel('s','','',306)
ansys.dl('all','','uy',0)
ansys.allsel('all')
ansys.tunif(T_UNI)
ansys.pbc('all','',1)
ansys.replot()

In [ ]:
Image(filename='{}/{}003.png'.format(mkpath,filecode),height=600,width = 600)

#### Write Physics 

In [ ]:
#/solu
ansys.slashsolu()
ansys.antype(4)
ansys.nropt('unsym')
ansys.kbc(0)
ansys.autots('on')
ansys.nlgeom('on')
ansys.neqit(100)
ansys.outres('all','all')
ansys.allsel('all')
ansys.physics('write','PHYS_STRUCT',PHYS_STRUCT)
ansys.physics('clear')
ansys.save()
ansys.finish()

### Thermal Electric Feld Setup
- Material
- Meshing
- Contact Elements
- Bondary Conditions
- Write Physics

#### Material 

In [ ]:
mat_read(1,mat_sheet,mat_path)
mat_read(2,mat_buckel,mat_path)
mat_read(3,mat_elektrode,mat_path)

In [ ]:
# a list to check mat
ansys.mplist('all')

In [ ]:
# ## delete mat if something wrong happened
# ansys.prep7()
# ansys.mpdele('all','all')
# ansys.tbdele('all','all')
# ansys.mptemp()

#### Meshing 

In [ ]:
ansys.et(1,223,110,'',1 )
ansys.allsel('all')

#### Contact Elements
- contact pair 1: ring - lower sheer

In [ ]:
ansys.prep7()
ansys.mat(2)
ansys.r(3)
ansys.real(3)
ansys.et(2,169)
ansys.et(3,172)
ansys.r(3,'',1.0,0.1,0,)
ansys.rmore('','',1.0E20,0.0,1.0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.rmore('','','','',)
ansys.rmore('','','',913)
ansys.keyopt(3,1,4)
ansys.keyopt(3,2,0)
ansys.keyopt(3,3,0)
ansys.keyopt(3,4,0)
ansys.keyopt(3,5,0)
ansys.keyopt(3,6,0)
ansys.keyopt(3,7,0)
ansys.keyopt(3,8,0)
ansys.keyopt(3,9,0)
ansys.keyopt(3,10,0)
ansys.keyopt(3,11,0)
ansys.keyopt(3,12,5)

- contact pair 2: upper electrode - sheer

In [ ]:
ansys.prep7()
ansys.mat(1)
ansys.r(5)
ansys.real(5)
ansys.et(4,169)
ansys.et(5,172)
ansys.r(5,'',1.0,0.1,0,)
ansys.rmore('','',1.0E20,0.0,1.0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.rmore('','','','',)
ansys.rmore('','','',913)
ansys.keyopt(5,1,4)
ansys.keyopt(5,2,0)
ansys.keyopt(5,3,0)
ansys.keyopt(5,4,0)
ansys.keyopt(5,5,0)
ansys.keyopt(5,6,0)
ansys.keyopt(5,7,0)
ansys.keyopt(5,8,0)
ansys.keyopt(5,9,0)
ansys.keyopt(5,10,0)
ansys.keyopt(5,11,0)
ansys.keyopt(5,12,5)

- contact pair 3: lower electrode - lower sheer

In [ ]:
ansys.prep7()
ansys.mat(3)
ansys.r(7)
ansys.real(7)
ansys.et(6,169)
ansys.et(7,172)
ansys.r(7,'',1.0,0.1,0,)
ansys.rmore('','',1.0E20,0.0,1.0,)
ansys.rmore(0.0,'',1.0,'',1.0,0.5)
ansys.rmore('',1.0,1.0,0.0,'',1.0)
ansys.rmore('','','','',)
ansys.rmore('','','',913)
ansys.keyopt(7,1,4)
ansys.keyopt(7,2,0)
ansys.keyopt(7,3,0)
ansys.keyopt(7,4,0)
ansys.keyopt(7,5,0)
ansys.keyopt(7,6,0)
ansys.keyopt(7,7,0)
ansys.keyopt(7,8,0)
ansys.keyopt(7,9,0)
ansys.keyopt(7,10,0)
ansys.keyopt(7,11,0)
ansys.keyopt(7,12,5)

#### Bondary Conditions 

In [ ]:
ansys.allsel('all')
ansys.nsel('s', 'loc', 'y', PROJECTION_H1 + ELECTRODE_H1)
ansys.cp(2, 'volt', 'all')
ansys.get('n1', 'node', '', 'num', 'min')
ansys.f('n1', 'amps', 0)
ansys.lsel('s', '', '', 306)
ansys.dl('all', '', 'volt', 0)
ansys.allsel('all')
ansys.tunif(T_UNI)
# force at node
ansys.flist('n1')
ansys.pbc('all', '', 1)
ansys.replot()

In [ ]:
Image(filename='{}/{}004.png'.format(mkpath,filecode),height=600,width = 600)

#### Write Physics 

In [ ]:
#/solu
ansys.slashsolu()
ansys.nlhist('nsol', '', 'temp', 'max', 0)
ansys.antype(4)
ansys.kbc(0)
ansys.autots('off')
ansys.nropt('unsym', '', 'off')
ansys.trnopt('full', '', '', '', '', 'hht')
ansys.lnsrch('on')
ansys.neqit(100)
ansys.outres('all', 'all')
ansys.allsel('all')
ansys.physics('write', 'PHYS_T_E', PHYS_T_E)
ansys.physics('clear')
ansys.save()
ansys.finish()

### Solution Mechanical Problem
- load strut physical enviroment
- divide mechanical solution into 4 parts
    - part1: Preloadling
    - part2: Spare keep constant force
    - part3: Applying welding force until start of current flow
    - part4: Applying welding force after start of current flow
- output pressure data for ECR-calcution

#### Load strut physical enviroment

In [ ]:
with ansys.non_interactive:
    ansys.run("/FILNAME,{},1".format(STRUCT_FILE))
    ansys.run("save")
ansys.assign('esav', PHYS_STRUCT, 'esav')
ansys.assign('emat', PHYS_STRUCT, 'emat')
ansys.input('{}/{}'.format(mkpath, PHYS_STRUCT), 'ph1')

#### Divide mechanical solution into 4 parts 
- TODO
    - Path kopieren und einfügen

In [ ]:
def fk_force(lsnum, kp, lab, force, endtime, dtime, dtmin, dtmax, kbc):
    ansys.slashsolu()
    ansys.antype(4)
    ansys.fk(kp, lab, force)
    ansys.sbctran()
    ansys.time(endtime)
    ansys.deltim(dtime, dtmin, dtmax)
    ansys.kbc(kbc)
    ansys.outres('all', 'all')
    ansys.allsel('all')
    ansys.lswrite(lsnum)

In [ ]:
# part1: Preloadling
fk_force(1,410,'fy',-1*IN_F_PRE,TIME_PF_ABS,DT_PF,DT_PF*0.1,DT_PF,0)
# part2: Spare keep constant force
fk_force(2,410,'fy',-1*IN_F_PRE,TIME_SWF_ABS,DT_SP,DT_SP,DT_SP,1)
# part3: Applying welding force until start of current flow
fk_force(3,410,'fy',-1*IN_F_IGN,TIME_SI_ABS,DT_WF_SI,DT_WF_SI*0.01,DT_WF_SI*100,0)
# part4: Applying welding force after start of current flow
fk_force(4,410,'fy',-1*IN_F_FIN,TIME_E_ABS,DT_I,DT_I,DT_I,0)

In [ ]:
# path kopieren
with ansys.non_interactive:
    ansys.run("lssolve,1,4,1")

In [ ]:
def follow(thefile):
    thefile.seek(0, 2)
    T_or_F = True
    while T_or_F:
        line = thefile.readlines()
        if not line:
            time.sleep(0.5)
            continue
        size1 = os.path.getsize("{}.out".format(file))
        time.sleep(3)
        size2 = os.path.getsize("{}.out".format(file))
        time.sleep(3)
        if (size1 == size2):
            T_or_F = False
        yield line, size1, size2
        

f = open(r'C:\Users\Simulation\AppData\Local\pyansys\pyansys\tmp_vxrhajockd.inp','r') # TODO kopierter Path hier einfügen
f.seek(10)
file = f.readline().split('.')[0]
logfile = open("{}.out".format(file), "r")
loglines = follow(logfile)
for tup in loglines:
    for line in tup[0]:
        print(line)
    if tup[1] == tup[2]:
        break

In [ ]:
ansys.post1()
ansys.pldisp(2)
Image(filename='{}/{}000.png'.format(mkpath,STRUCT_FILE),height=600,width = 600)

In [ ]:
ansys.save()

#### output pressure data for ECR-calcution
- select relevant nodes
- write the require paremeters in ansys apdl
- write out the pressure data  with time  and save as csv

##### select relevant nodes

In [ ]:
ansys.prep7()
ansys.allsel('all')
ansys.upgeom(1, 3, 'last', STRUCT_FILE, 'rst')

In [ ]:
ansys.post1()
ansys.lsel('s', '', '', 200, 202, 1)
ansys.nsll('s', 1)
ansys.esln('s', 0)
ansys.nsle('r', 'corner')
ansys.nplot()
ansys.load_parameters()
ansys.parameters

In [ ]:
Image(filename='{}/{}001.png'.format(mkpath,STRUCT_FILE),height=600,width = 600)

##### write the require paremeters in ansys apdl

In [ ]:
# define the name of the substeps 
ansys.post1()
par = ['NMBR_SS_PF', 'NMBR_SS_SP', 'NMBR_SS_SWF', 'NMBR_SS_EI']
ansys.run("ND_PRES_CUR = 0")
ansys.load_parameters()

# get the substeps number of eath Loadstep
for i in range(len(par)):
    ansys.set(i + 1, 'last')
    ansys.get(par[i], 'active', 0, 'solu', 'ncmss')
    ansys.load_parameters()

NMBR_SS = sum([ansys.parameters[ele] for ele in par])
NMBR_SS_EI = ansys.parameters['NMBR_SS_EI']

# count the number of node on contact surface
ansys.get('CNTCT1_ND_NMR', 'node', 0, 'count')
# get the max nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MAX', 'node', 0, 'num', 'max')
# get the min nodenumber of the selected nodes
ansys.get('CNTCT1_ND_MIN', 'node', 0, 'num', 'min')
# build a node time press tab
ansys.dim('CNTCT1_ND_PRES', 'table', 'CNTCT1_ND_NMR', NMBR_SS_EI, '', '', 'time')
ansys.load_parameters()
ansys.parameters

#####  write out the pressur data e with time and save as csv 
- durch for loop werden
    - min node Nummer == aktelle node Number
    - zuerst die Zeit von einem Substep gekregt
    - schreiben die zeit als columns von Tab
    - kommen zu einer anderen Loop
        - schreiben die aktuelle node Number ald index von Tab
        - kriegen Pressure von akutelle node Number
        - kriegen die nächste node Number
- Schreibe aus als csv Datei (*vwrite alle Daten und die zusammengefügte Daten als csv Datei speichern)
- Daten erkennen
- TODO: change name of dataframe

In [ ]:
time_col, node_index = [],[]
ansys.set(3,'last')
CNTCT1_ND_MIN = ansys.parameters['CNTCT1_ND_MIN']
CNTCT1_ND_NMR = ansys.parameters['CNTCT1_ND_NMR']
NMBR_SS_EI = ansys.parameters['NMBR_SS_EI']
for i in range(int(NMBR_SS_EI)):
    CNTCT1_ND_CUR = CNTCT1_ND_MIN
    ansys.get('TIME_LS', 'active', 0, 'set', 'time')
    ansys.load_parameters()
    TIME_LS = ansys.parameters['TIME_LS']
    time_col.append(TIME_LS)
    ansys.taxis('CNTCT1_ND_PRES(1,{})'.format(i+1),2,TIME_LS)
    for j in range(int(CNTCT1_ND_NMR)):
        if i == 0:
            ansys.taxis('CNTCT1_ND_PRES({},1)'.format(j+1),1,CNTCT1_ND_CUR) 
            node_index.append(CNTCT1_ND_CUR)
            ansys.get('CNTCT1_ND_PRES({},{})'.format(j+1,i+1),'node',CNTCT1_ND_CUR,'cont', 'pres')
            ansys.get('CNTCT1_ND_CUR', 'node', CNTCT1_ND_CUR, 'nxth')
            ansys.load_parameters()
            CNTCT1_ND_CUR = ansys.parameters['CNTCT1_ND_CUR']
        else:
            ansys.get('CNTCT1_ND_PRES({},{})'.format(j+1,i+1),'node',node_index[j],'cont', 'pres')
              
    ansys.set('next')

In [ ]:
a = list(np.arange(1,len(time_col)+1,1))
b = 0
files = []
TB = 'CNTCT1_ND_PRES'
for j in range(len(a)//10 + 1):
    ansys.cfopen('{}/{}/pres_{}'.format(mkpath, subfolder,j+1), 'csv')
    files.append('pres_{}.csv'.format(j+1))
    with ansys.non_interactive:
        if (j+1<=14):
            c =[a[i+b] for i in range(10)] 
            print(c)
            ansys.run(f'*VWRITE,{TB}(1,{c[0]}),{TB}(1,{c[1]}),{TB}(1,{c[2]}),{TB}(1,{c[3]}),{TB}(1,{c[4]}),{TB}(1,{c[5]}),{TB}(1,{c[6]}),{TB}(1,{c[7]}),{TB}(1,{c[8]}),{TB}(1,{c[9]})')
            ansys.run("(ES10.3,',',ES10.3,',',ES10.3,',',ES10.3,',',ES10.3,','ES10.3,',',ES10.3,',',ES10.3,',',ES10.3,',',ES10.3)")
            b +=10 
        
        else:
            ansys.run(f'*VWRITE,{TB}(1,{a[-3]}),{TB}(1,{a[-2]}),{TB}(1,{a[-1]})')
            ansys.run("(ES10.3,',',ES10.3,',',ES10.3)")
    time.sleep(1)

In [ ]:
test_all = pd.DataFrame()
for file in files:
    df = pd.read_csv('{}/{}/{}'.format(mkpath,subfolder,file), header=None)
    test_all = pd.concat([test_all,df],axis=1,)
test_all.columns = time_col
df_name = 'Press_Time'  # TODO
test_all[0] = node_index
test_all.set_index([0], inplace=True)
test_all.to_csv('{}/{}/{}.csv'.format(mkpath, subfolder, df_name))

#### Calculate electrical and thermal conductivity at 298 K

In [ ]:
# Beginn with calculation at sarting time of current flow
ansys.post1()
ansys.set(3, 'last')

In [ ]:
CNTCT1_ND_NMR = ansys.parameters['CNTCT1_ND_NMR']
ansys.dim('CNTCT1_ECC_X', 'table', CNTCT1_ND_NMR, 1, 1, 'x', 'ECC')
ansys.dim('CNTCT1_TCC_X', 'table', CNTCT1_ND_NMR, 1, 1, 'x', 'TCC')

In [ ]:
df5 = pd.read_csv('{}/{}/{}.csv'.format(mkpath,subfolder,df_name),index_col=0)
df5

In [ ]:
# Calculate electrical and thermal conductivity at 298 k
def table_save(dfname, list1, list2, tread=False, t_name=None):
    df = pd.DataFrame({0: list1, 1: list2}, index=None)
    df.sort_values(0, axis=0, ascending=True, inplace=True)
    df.to_csv('{}/{}/{}.csv'.format(mkpath, subfolder, dfname),
              index=None,
              header=None)
    if tread == True:
        ansys.finish()
        ansys.tread('CNTCT1_{}_X'.format(t_name),'{}/{}/{}'.format(mkpath, subfolder, dfname), 'csv')

ECR_l, ECC_l, TCC_l, NX_l= [], [], [], []
CNTCT1_ND_l = df5.index.tolist()
CNTCT1_ND_NMR = ansys.parameters['CNTCT1_ND_NMR']
CNTCT1_ND_MIN = ansys.parameters['CNTCT1_ND_MIN']
ansys.get('TIME_LS', 'active', 0, 'set', 'time')
ansys.load_parameters()
time = ansys.parameters['TIME_LS']
# compare yeild between two Mat at 298K
ansys.run('YIELD_MAT1 = TABLE_YIELD_MAT1({})'.format(T_UNI))
ansys.run('YIELD_MAT2 = TABLE_YIELD_MAT2({})'.format(T_UNI))
ansys.load_parameters()
yield_min = min(ansys.parameters['YIELD_MAT1'], ansys.parameters['YIELD_MAT2'])
# sum RSVX of two Mat at 298K
ansys.run('RSVX_MAT1 = TABLE_RSVX_MAT1({})'.format(T_UNI))
ansys.run('RSVX_MAT2 = TABLE_RSVX_MAT2({})'.format(T_UNI))
ansys.load_parameters()
RSVX = sum([ansys.parameters['RSVX_MAT1'], ansys.parameters['RSVX_MAT2']]) / 2
for i in range(len(CNTCT1_ND_l)):
    ansys.get('NX', 'node', CNTCT1_ND_l[i], 'loc', 'x')
    ansys.load_parameters()
    NX = ansys.parameters['NX']
    press = df5.loc[CNTCT1_ND_l[i], str(time)]
    if press == 0:
        ECR = np.nan
        ECC = 0
    else:
        ECR = 3 * 50 * CNTCT1_LYR_THCKNSS * ((yield_min / press)**CNTCT1_SRFC_CNDTN) * (RSVX)
        ECC = 1 / ECR
    TCC = LRNTZ_NMBR * ECC * T_UNI
    ECR_l.append(ECR)
    ECC_l.append(ECC)
    TCC_l.append(TCC)
    NX_l.append(NX)

table_save('ECR_X_{}'.format(time), NX_l, ECR_l)
table_save('ECC_X_{}'.format(time), NX_l, ECC_l, True, 'ECC')
table_save('TCC_X_{}'.format(time), NX_l, TCC_l, True, 'TCC')

In [ ]:
df = pd.read_csv('{}/{}/{}.csv'.format(mkpath,subfolder,'ECR_X_{}'.format(time)),header=None,sep = ',' ,engine='python',names = ['X','ECR'])
df

In [ ]:
ansys.allsel('all')
ansys.prep7()
ansys.upgeom(-1, 3, 'last', STRUCT_FILE, 'rst')
ansys.allsel('all')
ansys.parsav('all', 'PRMTR_DT', 'sav')
ansys.save()
ansys.finish()

In [ ]:
df6 = pd.read_csv('{}/{}/{}.csv'.format(mkpath, subfolder, 'Press_Time'),index_col=None)
df6.insert(0, 'X', NX_l)
df6.set_index(['X', '0'], inplace=True)
df6.sort_index(inplace=True)
df6 = df6.loc[(df6 != 0).any(axis=1), :]
df6.head()

In [ ]:
import matplotlib as mpl

x = [df6.index[i][0] for i in range(len(df6.index))]
time_v = [float(time) for time in list(df6)]
time_v_str = [str(time) for time in time_v]
norm_time = [(time - min(time_v)) / (max(time_v) - min(time_v))
             for time in time_v]
ticks = time_v[0::10]
n = len(list(df6))

# fig, ax = plt.subplots(figsize=(30, 15))
# colors = plt.cm.plasma_r(norm_time)
# norm = mpl.colors.Normalize(vmin=min(time_v), vmax=max(time_v))
# cmap = mpl.cm.ScalarMappable(norm=norm, cmap=plt.cm.plasma_r)
# cmap.set_array([])

# for idx, time in enumerate(list(df6)):
#     ax.plot(x, df6.loc[:, [time]] / 1e6, color=colors[idx], lw=0.8)

# ax.set_xlabel('X Koordi. in $m$', fontsize=17)
# ax.set_ylabel('Druck in $MPa$', fontsize=17)
# ax.set_ylim(0)
# cbar = plt.colorbar(cmap, ticks=ticks, drawedges=True, aspect=30)
# cbar.set_label('Zeit in  $s$', fontsize=15)
# cbar.ax.tick_params(labelsize=15,
#                     labelleft=True,
#                     labelright=False,
#                     direction='in',
#                     length=25)
# plt.tick_params(labelsize=17)
# title = 'Druckverteilung mit Position bei unterischielicher Zeitpunkt und 298K (unvollständig) '
# plt.title(title, fontsize=25)
# plt.grid()
# plt.show()

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
X, Y, Z = [], [], []
for a in range(len(x)):
    idx = a
    for b in range(n):
        idy = b
        X.append(x[idx] * 1000)
        Z.append(df6.iloc[idx, idy] / 1e6)
        Y.append(time_v[idy])
X = np.array(X).reshape(len(x), n)
Y = np.array(Y).reshape(len(x), n)
Z = np.array(Z).reshape(len(x), n)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X,
                       Y,
                       Z,
                       cmap=plt.cm.viridis_r,
                       alpha=0.8,
                       antialiased=False)
if min(time_v) == 0.1:
    title = 'Druckverteilung mit Position bei unterischielicher Zeitpunkt und 298K (vollständig) '
else:
    title = 'Druckverteilung mit Position bei unterischielicher Zeitpunkt und 298K (unvollständig) '
plt.title(title)
ax.set_xlabel('X Koordi. in $mm$')
ax.set_ylabel('Zeit in $s$')
ax.set_zlabel('Druck in $MPa$')
fig.colorbar(surf, shrink=0.5)
plt.show()

In [ ]:
SaveAndExit()